In [ ]:
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph import graph
from typing_extensions import TypedDict, List
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class MultiToolState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

internet_search_tool = TavilySearch()

def add(a:int, b:int) -> int:
    """Adds two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The sum of the two numbers.
    """
    return a + b    

def multiply(a:int, b:int) -> int:
    """Multiplies two numbers.

    Args:
        a (int): The first number.
        b (int): The second number.

    Returns:
        int: The product of the two numbers.
    """
    return a * b

tools = [internet_search_tool, add, multiply]

llm_with_tools = llm.bind_tools(tools)

def superbot(state: MultiToolState):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(MultiToolState)
graph_builder.add_node("superbot", superbot)
graph_builder.add_edge(START, "superbot")
graph_builder.add_edge("superbot", END) 

graph = graph_builder.compile()

res = graph.invoke({"messages": ["can you add 2 and 30, please?"]})

print(res)

#output 
#{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='a0ff2120-91e9-44ed-a17d-00073b033685'), AIMessage(content='2 + 30 = 32', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 46, 'total_tokens': 54, 'completion_time': 0.002070092, 'prompt_time': 0.011436672, 'queue_time': 0.001603759, 'total_time': 0.013506764}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2477e04561', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6a60b6cc-0ef9-4bba-b9f7-5bda29a79007-0', usage_metadata={'input_tokens': 46, 'output_tokens': 8, 'total_tokens': 54})]}

{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='a0ff2120-91e9-44ed-a17d-00073b033685'), AIMessage(content='2 + 30 = 32', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 46, 'total_tokens': 54, 'completion_time': 0.002070092, 'prompt_time': 0.011436672, 'queue_time': 0.001603759, 'total_time': 0.013506764}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2477e04561', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6a60b6cc-0ef9-4bba-b9f7-5bda29a79007-0', usage_metadata={'input_tokens': 46, 'output_tokens': 8, 'total_tokens': 54})]}


In [9]:
def superbot(state: MultiToolState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder = StateGraph(MultiToolState)
graph_builder.add_node("superbot", superbot)
graph_builder.add_edge(START, "superbot")
graph_builder.add_edge("superbot", END) 

graph = graph_builder.compile()

res = graph.invoke({"messages": ["can you add 2 and 30, please?"]})

print(res)

#output
#{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='82e6eecd-d7ac-46f1-bf2a-a51f35470efc'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rmwn2k9n8', 'function': {'arguments': '{"a":2,"b":30}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1925, 'total_tokens': 1943, 'completion_time': 0.050792134, 'prompt_time': 0.155359789, 'queue_time': 0.068414258, 'total_time': 0.206151923}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_a2c262bc3a', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--acf11271-23f8-4aa1-9248-593420e17c23-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 30}, 'id': 'rmwn2k9n8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1925, 'output_tokens': 18, 'total_tokens': 1943})]}

{'messages': [HumanMessage(content='can you add 2 and 30, please?', additional_kwargs={}, response_metadata={}, id='f7dfa87c-407e-4e6d-af25-be4e11115a13'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'b5jwn0hkp', 'function': {'arguments': '{"a":2,"b":30}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1925, 'total_tokens': 1943, 'completion_time': 0.05078824, 'prompt_time': 0.166654531, 'queue_time': 0.003068357, 'total_time': 0.217442771}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_f089a29207', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2b68b725-3cd5-443f-a8ce-328835100c70-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 30}, 'id': 'b5jwn0hkp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1925, 'output_tokens': 18, 'total_tokens': 1943})]}
